<a href="https://colab.research.google.com/github/Jeet009/legal-assistant-qdrant/blob/main/legal_assistant_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q qdrant-client transformers torch pdfplumber tqdm sentencepiece fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 26.8 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adarshsingh0903/legal-dataset-sc-judgments-india-19502024")

print("Path to dataset files:", path)

  6%|▋         | 423M/6.40G [00:05<01:16, 84.1MB/s]


KeyboardInterrupt: 

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from transformers import AutoTokenizer, AutoModel
import pandas as pd, os, re
from tqdm import tqdm
import json
import torch
import pdfplumber
# import fitz  # PyMuPDF

# if using Qdrant Cloud: replace with your URL + API key
qdrant = QdrantClient(
    url="",
    api_key=""
)


### Building the Legal Knowledge Base

In [ ]:
BASE_DIR = "/root/.cache/kagglehub/datasets/adarshsingh0903/legal-dataset-sc-judgments-india-19502024/versions/1/supreme_court_judgments/2025"

# Qdrant collection name
COLLECTION_NAME = "indian_kanoon_2025"

# Model name (legal BERT)
MODEL_NAME = "law-ai/InLegalBERT"

# Chunking params
MAX_TOKENS = 512
STRIDE = 256  # 50% overlap

# Batch size for Qdrant upserts
UPSERT_BATCH = 64

# device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

Using device: cpu


In [ ]:
print("Loading model:", MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

# mean pooling helper
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]   # (batch, seq_len, hidden)
    mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * mask_expanded, dim=1)
    sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask


Loading model: law-ai/InLegalBERT


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

In [ ]:
# Ensure collection exists with correct vector size (BERT base -> 768)
VECTOR_SIZE = model.config.hidden_size  # typically 768
print("Vector size:", VECTOR_SIZE)

# Recreate collection for a fresh demo (CAUTION: destroys existing data in the same collection)
qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
)
print("Created Qdrant collection:", COLLECTION_NAME)


Vector size: 768


/tmp/ipython-input-3295741246.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


Created Qdrant collection: indian_kanoon_2025


In [ ]:
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_txt = page.extract_text()
                if page_txt:
                    text += page_txt + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

def tokenize_ids(text):
    # returns list of token ids (no special tokens)
    return tokenizer.encode(text, add_special_tokens=False)

def detokenize(token_ids):
    return tokenizer.decode(token_ids, clean_up_tokenization_spaces=True)

def chunk_tokens_to_texts(token_ids, max_len=MAX_TOKENS, stride=STRIDE):
    chunks = []
    n = len(token_ids)
    if n == 0:
        return []
    for start in range(0, n, (max_len - stride)):
        end = min(start + max_len, n)
        chunk = token_ids[start:end]
        chunks.append(detokenize(chunk))
        if end == n:
            break
    return chunks

def parse_case_metadata(pdf_path):
    # derive year from parent folder and case name from filename
    folder = os.path.basename(os.path.dirname(pdf_path))
    try:
        year = int(folder)
    except:
        year = None
    fname = os.path.basename(pdf_path)
    case_name = os.path.splitext(fname)[0].replace("_", " ")
    return year, case_name


In [ ]:
def embed_text_chunk(text):
    # tokenize + produce embedding for a text (<=512 tokens)
    inputs = tokenizer(text, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        emb = mean_pooling(outputs, inputs["attention_mask"])
    # return 1D numpy array
    return emb[0].cpu().numpy()


In [ ]:
from uuid import uuid4

pdf_files = []
for fname in os.listdir(BASE_DIR):
    if fname.lower().endswith(".pdf"):
        pdf_files.append(os.path.join(BASE_DIR, fname))

print("Found PDFs in 2025 folder:", len(pdf_files))

points_buffer = []
total_chunks = 0

for pdf_path in tqdm(pdf_files[:10], desc="Processing 2025 PDFs"):  # limit to 10 for testing
    year, case_name = parse_case_metadata(pdf_path)
    raw_text = extract_text_from_pdf(pdf_path)
    if not raw_text or len(raw_text) < 50:
        continue

    # tokenize + chunk
    token_ids = tokenize_ids(raw_text)
    text_chunks = chunk_tokens_to_texts(token_ids, max_len=MAX_TOKENS, stride=STRIDE)

    for idx, chunk_text in enumerate(text_chunks):
        # ✅ generate valid UUID for each chunk
        point_id = str(uuid4())

        emb = embed_text_chunk(chunk_text)
        payload = {
            "year": year,
            "case_name": case_name,
            "court": "Supreme Court of India",
            "jurisdiction": "India",
            "source_path": pdf_path,
            "chunk_index": idx,
            "file_name": os.path.basename(pdf_path),
            "text_preview": chunk_text[:1000]
        }
        point = PointStruct(id=point_id, vector=emb.tolist(), payload=payload)
        points_buffer.append(point)
        total_chunks += 1

        # batch upsert
        if len(points_buffer) >= UPSERT_BATCH:
            qdrant.upsert(collection_name=COLLECTION_NAME, points=points_buffer)
            points_buffer = []

# final flush
if points_buffer:
    qdrant.upsert(collection_name=COLLECTION_NAME, points=points_buffer)

print(f"✅ Done. Upserted {total_chunks} chunks successfully.")


Found PDFs in 2025 folder: 400


Processing 2025 PDFs: 100%|██████████| 10/10 [10:45<00:00, 64.52s/it]


✅ Done. Upserted 338 chunks successfully.


### Implementing Semantic Search with Qdrant

In [ ]:
query = "cases related to murder"
# Use the mean_pooling function to get the query embedding
inputs = tokenizer(query, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt")
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    query_vec = mean_pooling(outputs, inputs["attention_mask"])[0].cpu().numpy().tolist()


hits = qdrant.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vec,
    limit=5
)

for h in hits:
    print("Score:", h.score)
    print("Case name:", h.payload["case_name"])
    print("Snippet:", h.payload["text_preview"][:400], "\n---\n")

/tmp/ipython-input-3741072256.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


Score: 0.6233009
Case name: Radhika Agarwal vs Union Of India on 27 February 2025 1
Snippet: in adri dharan das vs. state of w. b. 4, ordinarily arrest is a part of the process of investigation intended to secure several purposes. the accused may have to be questioned in detail regarding various facets of motive, preparation, commission and aftermath of crime and the connection of other persons, if any, in the crime. there may be circumstances in which the accused may provide information  
---

Score: 0.6161155
Case name: Ayyub vs State Of Uttar Pradesh on 7 February 2025 1
Snippet: being present along with r - 2 vijay, whereas in the statements mrs. sunesh also claims to have been present. 8. it further transpires from the application for quash that based on the information given by the ward boy of the hospital on 02. 11. 2022, the same was registered at police station g. d. 37 at 01 : 14 pm and then panchayatnama of deceased tanu was conducted at the hospital in the presenc 
---

Sco

#### Filtering using Qdrant PayLoad

In [ ]:
from qdrant_client.models import PayloadSchemaType

qdrant.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="jurisdiction",
    field_schema=PayloadSchemaType.KEYWORD,
)

qdrant.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="year",
    field_schema=PayloadSchemaType.INTEGER,
)

qdrant.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="case_name",
    field_schema=PayloadSchemaType.TEXT,  # use TEXT for substring/semantic match
)


UpdateResult(operation_id=23, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue, MatchText
# Use these values to check filtering output
# Ayyub vs State Of Uttar Pradesh on 7 February 2025 1
# Radhika Agarwal vs Union Of India on 27 February 2025 1
filter_condition = Filter(
    must=[
        FieldCondition(key="jurisdiction", match=MatchValue(value="India")),
        FieldCondition(key="year", match=MatchValue(value=2025)),
        FieldCondition(key="case_name", match=MatchText(text="Ayyub"))
    ]
)

In [ ]:
inputs = tokenizer(query, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    outputs = model(**inputs)
    query_vec = mean_pooling(outputs, inputs["attention_mask"])[0].cpu().numpy().tolist()

hits = qdrant.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vec,
    limit=1,
    query_filter=filter_condition,
    with_payload=True
)

for h in hits.points:
    print(f"Score: {h.score:.4f}")
    print(h.payload["case_name"])
    print(h.payload["text_preview"][:200])
    print("---")


Score: 0.6161
Ayyub vs State Of Uttar Pradesh on 7 February 2025 1
being present along with r - 2 vijay, whereas in the statements mrs. sunesh also claims to have been present. 8. it further transpires from the application for quash that based on the information give
---


In [ ]:
# TODO - Hybrid Search

### Legal Reasoning with LLM Integration

In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
from groq import Groq
import textwrap

In [ ]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "")
GROQ_MODEL = "llama-3.3-70b-versatile"

groq = Groq(api_key=GROQ_API_KEY)

In [ ]:
# ---------- Helper: retrieve top-k from Qdrant ----------
def retrieve_top_k(query_vector, k=8, filter_condition=None, with_payload=True):
    """
    Uses Qdrant query_points (dense-only). Assumes collection stores 'dense' vectors as default.
    query_vector: list[float] (embedding)
    filter_condition: qdrant_client.models.Filter or None
    """
    resp = qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=k,
        query_filter=filter_condition,
        with_payload=with_payload
    )
    # resp.points is a list of point objects
    return resp.points

In [ ]:
# ---------- Helper: assemble context (token budget aware approximation) ----------
def build_rag_context(points, char_limit=6000):
    """
    Concatenate retrieved chunks into a numbered context block.
    Respect char_limit (approx safe to avoid huge prompts). Use text_preview payload.
    Returns context string and list of source metadata tuples.
    """
    parts = []
    sources = []
    chars = 0
    for i, p in enumerate(points):
        payload = p.payload or {}
        preview = payload.get("text_preview") or payload.get("chunk_text") or ""
        # short label
        case_name = payload.get("case_name", payload.get("file_name", "unknown"))
        year = payload.get("year", "")
        src = f"{case_name} ({year})"
        snippet = preview.strip()
        entry = f"[{i+1}] {src}\n{snippet}\n"
        if chars + len(entry) > char_limit:
            break
        parts.append(entry)
        sources.append({"index": i+1, "case_name": case_name, "year": year, "source_path": payload.get("source_path")})
        chars += len(entry)
    context = "\n".join(parts)
    return context, sources

In [ ]:
# ---------- Prompt template ----------
PROMPT_TEMPLATE = textwrap.dedent("""
You are an expert legal research assistant. Using ONLY the CONTEXT below, draft a 3-paragraph summary that answers the USER QUESTION.
- Each paragraph should be short (3-5 sentences).
- Include relevant case names (with the [n] citation referencing the CONTEXT item) next to the sentences they support.
- Use plain English and explain the legal reasoning and relevant holdings.
- If the context is insufficient, explicitly say you could not find enough authoritative context and suggest filters or search terms.

CONTEXT:
{context_block}

USER QUESTION:
{user_question}

INSTRUCTIONS:
1) Provide a clear 3-paragraph answer with citations (e.g., \"... this suggests proportionality is required (see [2])\").
2) Then give a short \"Sources\" list that maps [n] to the case name and source path.
""").strip()

In [ ]:
# ---------- Main RAG function ----------
def rag_answer(user_question, query_vector, qdrant_filter=None, top_k=12, char_limit=6000, groq_model=GROQ_MODEL):
    # 1) Retrieve
    points = retrieve_top_k(query_vector=query_vector, k=top_k, filter_condition=qdrant_filter, with_payload=True)
    if not points:
        return {"error": "No retrieval results from Qdrant for the given query/filter."}
    # 2) Build context (truncate to char_limit)
    context_block, sources = build_rag_context(points, char_limit=char_limit)
    if not context_block.strip():
        return {"error": "Retrieved docs are too large after truncation or empty previews."}
    # 3) Format prompt
    prompt = PROMPT_TEMPLATE.format(context_block=context_block, user_question=user_question)
    system_message = {"role": "system", "content": "You are a helpful and cautious legal research assistant. Cite sources exactly as requested."}
    user_message = {"role": "user", "content": prompt}
    messages = [system_message, user_message]
    # 4) Call Groq Chat Completions
    # Use conservative temperature; set max tokens reasonably.
    chat_completion = groq.chat.completions.create(
        messages=messages,
        model=groq_model,
        temperature=0.0,
        max_completion_tokens=800
    )
    # 5) Extract result
    llm_text = chat_completion.choices[0].message.content
    # 6) Return structured object
    return {
        "answer": llm_text,
        "sources": sources,
        "raw_groq_response": chat_completion
    }

In [ ]:
query = "Summarize Supreme Court judgments on Ayyub vs State of UP"
inputs = tokenizer(query, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    outputs = model(**inputs)
    query_vec = mean_pooling(outputs, inputs["attention_mask"])[0].cpu().numpy().tolist()

In [ ]:
res = rag_answer("Summarize Supreme Court judgments on Ayyub vs State of UP", query_vector=query_vec, qdrant_filter=filter_condition)
print(res["answer"])

The Supreme Court of India has recently ruled on the case of Ayyub vs State of Uttar Pradesh, which involves the applicability of Section 306 of the Indian Penal Code (IPC) to the accused [1]. The Court has reiterated the principles that in cases of abetment of suicide, there must be proof of direct or indirect acts of incitement to commit suicide, as seen in cases such as Amalendu Pal alias Jhantu vs. State of West Bengal, (2010) 1 SCC 707, and M. Mohan vs. State, (2011) 3 SCC 626 [2]. This suggests that the Court requires specific evidence of abetment to establish the offence under Section 306 IPC.

The Court has also emphasized that the intention of the accused to aid or instigate the deceased to commit suicide is a crucial element in attracting Section 306 IPC, as held in Madan Mohan vs. State of M.P. and another, (1995 Supp (3) SCC 438) [3]. In the present case, the Court found that the utterance attributed to the appellants, even if assumed to be true, cannot be said to be of suc